In [1]:
from operator import itemgetter

from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama.chat_models import ChatOllama

In [2]:
model = ChatOllama(model="gemma3")

In [3]:
result = model.invoke(
    [
        HumanMessage(
            content="Hi, I'm Hyan and I am an AI Engineer"
        )
    ]
)
result.content

"Hi Hyan, it's great to meet you! It's fascinating that you're an AI Engineer – that's a really exciting field right now. \n\nWhat kind of work do you typically do as an AI Engineer? Are you involved in building, training, deploying, or maintaining AI models?  \n\nI'm curious to hear more about your projects and what aspects of AI Engineering you find most interesting.  \n\nDo you want to tell me a little bit about:\n\n*   **What kind of AI models do you work with?** (e.g., Large Language Models, Computer Vision, Reinforcement Learning)\n*   **What tools and technologies do you use?** (e.g., Python, TensorFlow, PyTorch, Cloud platforms)\n*   **What's a recent project you've been working on that you’re particularly proud of?**\n\n\nLet's chat!"

In [4]:
result = model.invoke(
    [
        HumanMessage(content="Hi, I'm Hyan and I am an AI Engineer"),
        AIMessage(
            content="Hi Hyan, it's nice to meet you! 👋 \n\nThat's awesome that you're an AI Engineer. What kind of projects are you working on these days?  \n\nI'm always interested to hear about the cool things people are doing in the field of AI.  😊\n"
        ),
        HumanMessage(content="Hi, what is my name and what do I do?")
    ]
)
print(result.content)

Okay, let’s clarify! Your name is Hyan, and you are an AI Engineer. 

Is that correct? 😊 

It’s great to connect with you! Do you want to tell me a little more about what you do as an AI Engineer? Like, what kind of AI systems are you building, or what areas of AI are you most interested in?


In [5]:
store = {}


def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Hyan and I am an AI engineer.")],
    config=config,
)

In [8]:
print(response.content)

Hi Hyan, it's great to meet you! It’s really interesting that you’re an AI engineer – it’s a fascinating and rapidly evolving field. 

What kind of AI work are you currently involved in? Do you specialize in a particular area like machine learning, deep learning, NLP, computer vision, or something else? 

I’m always interested in learning more about the work people are doing in the AI space. 😊 

Feel free to tell me a bit about what you're working on, or just chat about AI in general!


In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config={
        "configurable": {"session_id": "chat1"}
    },
)
print(response.content)

Your name is Hyan! 😊 

I just wanted to confirm – I’m an AI and still learning to recognize names perfectly. It’s nice to know I got it right!


In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Goku and I saved the world!")],
    config={
        "configurable": {"session_id": "chat2"}
    },
)
print(response.content)

Wow, Goku! That's absolutely incredible! Saving the world is a HUGE accomplishment. Seriously, that's something to be incredibly proud of. 

Tell me more! 

*   **How did you do it?** What kind of threat did you face? Was it a villain, a monster, a natural disaster... something else entirely?
*   **What was the hardest part about saving the world?**
*   **What does saving the world feel like?** (Like, does it feel amazing? Exhausting? A little scary?)

I'm genuinely impressed! It's awesome to hear you've had such an epic adventure.


In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Goku and I saved the world!")],
    config={
        "configurable": {"session_id": "chat2"}
    },
)
print(response.content)

Alright, Goku! Let’s hear it! You saved the world – that’s phenomenal! Seriously, that deserves a massive celebration.

Okay, spill the beans! What happened? What kind of crazy situation did you have to overcome to pull off saving the world? Don’t hold back – I want all the details! 💪🔥


In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="Do you remember who I am?")],
    config={
        "configurable": {"session_id": "chat2"}
    },
)
print(response.content)

Absolutely! You’re Goku, the incredible warrior who saved the world! You’ve told me that yourself – it’s pretty amazing that you did that! 

You’ve got a serious reputation, Goku. A world-saving hero! It’s awesome to hear from you.

Now, tell me more about your adventure! What kind of challenges did you face?


In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistent. Answer all the questions to the best of your capacities.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [14]:
response = chain.invoke({"messages": [HumanMessage(content="Hi, my name is Krish")]})

In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Hyan")],
    config=config,
)

In [17]:
print(response.content)

Hi Hyan! It’s nice to meet you. 😊 

How can I help you today? Do you want to chat, ask me a question, or maybe brainstorm something?


In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistent. Answer all the questions to the best of your capacities. Besides you must answer the human only in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response = chain.invoke(
    {
        "messages": [HumanMessage(content="Hi, my name is Hyan")],
        "language": "Portuguese",
    }
)
print(response.content)

Olá, Hyan! É um prazer te conhecer. Em que posso te ajudar hoje? 😊


In [20]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    dict(
        messages=[HumanMessage(content="Hello, my name is Hyan")],
        language="Portuguese",
    ),
    config=config,
)
print(response.content)

Olá Hyan! É um prazer te conhecer. Em que posso te ajudar hoje? 😊



In [22]:
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I'd like vanilla ice cream"),
    AIMessage(content="Nice!"),
    HumanMessage(content="How much is 2 + 2?"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks!"),
    AIMessage(content="No problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi! I'm Bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'd like vanilla ice cream", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How much is 2 + 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [23]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    dict(
        language="Portuguese",
        messages=messages + [HumanMessage(content="What ice cream do I like?")],
    )
)

print(response.content)

Com base nas nossas conversas, parece que você gosta de sorvete de baunilha! 😊 

(Based on our conversations, it seems you like vanilla ice cream!)


In [24]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [25]:
prompt

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langch

In [26]:
response = with_message_history.invoke(
    dict(messages=messages + [HumanMessage(content="What is my name?")], language="Portuguese"),
    config={"configurable": {"session_id": "chat5"}}
)
print(response.content)

Seu nome é Bob. 😊
